In [2]:
import anthropic
import json
import os
from dotenv import load_dotenv
import re
import pandas as pd

In [3]:
# Load environment variables
load_dotenv()

client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

In [4]:
def get_user_input():
    """Gather user input for the running plan."""
    fitness_level = input("What is your current fitness level? (Beginner/Intermediate/Advanced): ")
    goal = input("What is your running goal? (e.g., 5K, 10K, Half Marathon, Marathon): ")
    weeks = input("How many weeks do you want to train? (e.g., 8, 12, 16): ")
    days_per_week = input("How many days per week can you commit to running? (e.g., 3, 4, 5): ")
    time_goal = input("Do you have any specific time goals? (If yes, please specify): ")
    recent_race = input("Have you raced recently? If so please specify the distance and time: ")
    units = input("Would you prefer to use miles or kilometers? ")

    return {
        "fitness_level": fitness_level,
        "goal": goal,
        "weeks": weeks,
        "days_per_week": days_per_week,
        "time_goal": time_goal,
        "recent_race": recent_race,
        "units": units
    }

In [5]:
def generate_running_plan(user_input):
    """Generate a running plan using the Claude API."""
    system_prompt = "You are an AI assistant tasked with creating personalized running training plans based on user responses to a set of questions. Your goal is to generate a comprehensive, tailored plan that meets the user's specific needs and goals."
    prompt = f"""
    Here are the user's responses to the questionnaire:

    A {user_input['weeks']}-week running plan for a {user_input['fitness_level']} runner 
    training for a {user_input['goal']} race. The plan should include {user_input['days_per_week']} 
    running days per week. Their time goal is {user_input['time_goal']}. 
    A recent race result is {user_input['recent_race']}, ignore this if it is not specified. 
    Set the distances in {user_input['units']}

    Please provide a week-by-week breakdown of the training plan, including:
    1. Weekly mileage
    2. Types of runs (e.g., easy runs, long runs, speed work)
    3. Specific workouts
    4. Rest and cross-training days
    5. Gradual increase in intensity and volume
    6. Tapering period before the race

    Also, include general advice on nutrition, hydration, and injury prevention.

    Generate the training plan in JSON format, structured as follows:
    
    <json_structure>  
    {{
        "user_info": {{
            "fitness_level": "",
            "goal": "",
            "recent_race": {{
                "distance": "",
                "time": ""
            }},
            "training_duration_weeks": 0,
            "days_per_week": 0,
            "time_goal": "",
            "preferred_unit": ""
        }},
        "training_plan": [
            {{
                "week": 1,
                "workouts": [
                    {{
                        "day": 1,
                        "type": "",
                        "distance": "",
                        "description": ""
                    }}
                    // ... (repeat for each workout day)
                ]
            }}
            // ... (repeat for each week)
        ]
    }}
    </json_structure>
    
    For the \"type\" field in each workout, use one of the following categories:
    - Easy Run
    - Long Run
    - Tempo Run
    - Interval Training
    - Hill Repeats
    - Recovery Run
    - Cross-Training
    - Rest Day

    Ensure that the plan follows these guidelines:
    1. Gradually increase weekly mileage/distance
    2. Include a mix of workout types appropriate for the user's goal and fitness level
    3. Incorporate rest days and recovery runs to prevent overtraining
    4. For longer training plans (12+ weeks), include a tapering period before the goal race

    If the user has provided insufficient information or if there are any inconsistencies in their responses, make reasonable assumptions based on standard running training principles. For example, if a beginner aims for a marathon in 8 weeks, adjust the goal to a more realistic target like a 10K.

    Remember to maintain consistency in the unit of measurement (miles or kilometers) throughout the plan, as per the user's preference.

    Your final output should be valid JSON that can be easily parsed and used in web applications or downloaded by the user. Ensure that all fields are properly filled and the structure is maintained.
    """

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0,
        system=system_prompt,
        messages=[        
            {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
            }
        ]
    )

    return response.content



In [37]:
user_input = get_user_input()

In [38]:
running_plan = generate_running_plan(user_input)

In [39]:
print(running_plan[0].text)

Based on the user's responses, I've created a personalized 8-week marathon training plan for a beginner runner. Given the short timeframe and beginner status, I've adjusted the goal to focus on completing the marathon rather than aiming for a specific time. Here's the training plan in JSON format:

{
  "user_info": {
    "fitness_level": "beginner",
    "goal": "complete marathon",
    "recent_race": {
      "distance": "N/A",
      "time": "N/A"
    },
    "training_duration_weeks": 8,
    "days_per_week": 4,
    "time_goal": "completion",
    "preferred_unit": "kilometers"
  },
  "training_plan": [
    {
      "week": 1,
      "workouts": [
        {
          "day": 1,
          "type": "Easy Run",
          "distance": "5 km",
          "description": "Start with a comfortable pace you can maintain throughout the run."
        },
        {
          "day": 3,
          "type": "Interval Training",
          "distance": "6 km",
          "description": "5 x 400m at 5K pace with 90 s

In [40]:
text_block = running_plan[0].text

# Use regular expression to extract the JSON block
json_match = re.search(r'\{.*\}', text_block, re.DOTALL)
json_text = json_match.group(0)

# Split the text into parts: before JSON, JSON, and after JSON
before_json = text_block[:json_match.start()].strip()
after_json = text_block[json_match.end():].strip()


In [41]:
data = json.loads(json_text)

# Extract user info and training plan
user_info = data['user_info']
training_plan = data['training_plan']

In [1]:
print(data)

NameError: name 'data' is not defined

In [42]:
# Convert user info to a DataFrame
user_info_df = pd.DataFrame([user_info])

# Convert training plan to a DataFrame
training_plan_list = []
for week in training_plan:
    week_num = week['week']
    for workout in week['workouts']:
        workout['week'] = week_num
        training_plan_list.append(workout)
training_plan_df = pd.DataFrame(training_plan_list)

# Display the non-JSON text
print(before_json)
print("\n")

# Display user info table
print("User Information:")
print(user_info_df.to_string(index=False))
print("\n")

# Display training plan table
print("Training Plan:")
print(training_plan_df.to_string(index=False))
print("\n")

# Display the non-JSON text after the JSON block
print(after_json)

Based on the user's responses, I've created a personalized 8-week marathon training plan for a beginner runner. Given the short timeframe and beginner status, I've adjusted the goal to focus on completing the marathon rather than aiming for a specific time. Here's the training plan in JSON format:


User Information:
fitness_level              goal                        recent_race  training_duration_weeks  days_per_week  time_goal preferred_unit
     beginner complete marathon {'distance': 'N/A', 'time': 'N/A'}                        8              4 completion     kilometers


Training Plan:
 day              type distance                                                                          description  week
   1          Easy Run     5 km                   Start with a comfortable pace you can maintain throughout the run.     1
   3 Interval Training     6 km  5 x 400m at 5K pace with 90 sec rest between intervals. 1 km warm-up and cool-down.     1
   5          Easy Run     5 